In [30]:
# last modified 2024/4/24 
# %matplotlib ipympl
%matplotlib inline
import h5py 
import glob
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from DasTools import DasPrep as dp
import os
import datetime
import matplotlib
import logging
import csv
import time
def concat(flist , start = 0 , span = 20  ):
    datalist = []
    for fname in flist[ start : start + span ]:
        data = dp.read_das(fname)
        datalist.append(data)

    data = np.concatenate(datalist, axis = 1)
    utc_datetime = datetime.datetime.strptime(flist[start][-22 : -7],'%Y%m%d_%H%M%S') + datetime.timedelta(hours=+8)
    utc_day = utc_datetime.strftime("%m-%d %H:%M")
    print("cat")
    return data , utc_datetime, utc_day 


def psd(startn,endn):
    # 创建一个dataset
    span_each_pic = 10 #12h -> 50min
    concat_time = 0
    psd_time = 0
    all_start = time.time()

    ch_list = [20*i for i in range(148)]
    for n in range(startn, endn):
        data , utc_datetime, utc_day = concat(flist , start= span_each_pic * n ,span= span_each_pic)
        nfft = 30000
        fs = 2000
        start = time.time()


        for i in range(len(ch_list)):
            with h5py.File('../output/DAS/psd_5_11/test2_ch'+str(ch_list[i])+'.hdf5', 'a') as f:
                # g =  f.create_group(str(ch_list[i]))  
                [f1,Pxx1] = signal.welch(data[i],                   # 随机信号
                                nfft=nfft,               # 每个窗的长度
                                fs=fs,                   # 采样频率   
                                # detrend='mean',          # 去掉均值
                                window=np.hanning(nfft), # 加汉尼窗
                                noverlap=int(nfft*3/4),  # 每个窗重叠75%的数据
                                )        # 求单边谱
                # print(f1)
                f.create_dataset(str(utc_day),  data=Pxx1 , chunks=len(Pxx1))
                f.close()

        concat_time +=  time.time() -start
        all_time = time.time() -all_start
        print(n , "psd运行时间:%.2f min %d s"%( int(concat_time / 60) , concat_time%60)+"/ all :%.2f min %d s"%( int(all_time / 60) , all_time%60) , utc_day, "check in ")

flist = glob.glob('../../czyc/test3/*357.h5')
flist.sort()
print(flist[0] ,flist[-1])

# path = '../output/DAS/psd_5_11/'
# if not os.path.exists(path):
#     os.makedirs(path)

# print(int(len(flist)/10))

# flist = flist[:]

# psd(startn=642 , endn=667) #10 chunk 14min 
# #642-1204 psd运行时间:157.00 min 1 s/ all :617.00 min 50 s 01-01 20:45 check in 

# 537

../../czyc/test3/czyc_3km_GL_10m_frq_1kHz_sp_2m_UTC_20240103_140512.357.h5 ../../czyc/test3/czyc_3km_GL_10m_frq_1kHz_sp_2m_UTC_20240105_061712.357.h5


log
5.9 周四 罗老师和袁师兄
1.das Hz time 在30 40 50道 1.5晚上0-9点 低频部分有类似多普勒效应的能量，可以做stft放大清晰度。实现提示：十倍十倍地放大缩小窗长 以提高清晰度，重叠50%，
2.das Hz time 在30 40 50道 能量暴增前期  有一个有类似多普勒效应的能量，可以做stft放大清晰度。实现提示：十倍十倍地放大缩小窗长 以提高清晰度，重叠50%，

3.psd 主要是反应平稳信号 去掉噪声
4. 740道有局部异常，1.5 8-10点的信号有暴增，可以往前多画几天看看是否是规律人类活动，但是发现传播方向既不是

In [2]:
import DasBatch as db
import glob
import os

# flist = glob.glob('../../czyc/test3/*357.h5')
# flist.sort()
# print(len(flist))

# path = '../output/DAS/stft_5_12/'
# if not os.path.exists(path):
#     os.makedirs(path)

# index_start =flist.index('../../czyc/test3/czyc_3km_GL_10m_frq_1kHz_sp_2m_UTC_20240104_070012.357.h5')
# index_end =flist.index('../../czyc/test3/czyc_3km_GL_10m_frq_1kHz_sp_2m_UTC_20240104_160012.357.h5')
# print(flist[0] , index_start , index_end)

# # index_end =flist.index('../../czyc/test3/czyc_3km_GL_10m_frq_1kHz_sp_2m_UTC_20240105_010012.357.h5')

# span_each_pic = 20
# print(index_start,index_end )

# ch_list = [20+i for i in range(40)]
# db.das_time2ch(flist , ch_list , 
#                index_start ,  index_end  ,step= 10 , 
#                save_path= "../output/DAS/das_as_channel_5_14/test3_ch ") #大概30min


flist = glob.glob('../output/DAS/das_as_channel_5_14/test3_ch*')
flist.sort()
print(len(flist))

path = "../output/DAS/das_as_channel_5_14/fig5/"
if not os.path.exists(path):
    os.makedirs(path)


# db.cal_and_show_stft(flist= flist ,
#                   title = "czyc",
#                   save_path = path,
#                   fs=1000 ,  nperseg= 1024 , overlap= 512) #大概

db.cal_and_show_stft_split(flist= flist[::3] ,
                  title = "czyc",
                  save_path = "../output/DAS/das_as_channel_5_14/fig9/",
                  split = 18,
                  figsize0= [30,12],
                  fs=1000 ,  nperseg= 512 , overlap= 256) #大概


40
test3_ch 20   (54, 253124) (432,)
运行时间 all :0.00 min 42 s
test3_ch 23   (54, 253124) (432,)
运行时间 all :1.00 min 27 s
test3_ch 26   (54, 253124) (432,)
运行时间 all :1.00 min 56 s
test3_ch 29   (54, 253124) (432,)
运行时间 all :2.00 min 27 s
test3_ch 32   (54, 253124) (432,)
运行时间 all :2.00 min 53 s
test3_ch 35   (54, 253124) (432,)
运行时间 all :3.00 min 18 s
test3_ch 38   (54, 253124) (432,)
运行时间 all :3.00 min 44 s
test3_ch 41   (54, 253124) (432,)
运行时间 all :4.00 min 10 s
test3_ch 44   (54, 253124) (432,)
运行时间 all :4.00 min 37 s
test3_ch 47   (54, 253124) (432,)
运行时间 all :5.00 min 2 s
test3_ch 50   (54, 253124) (432,)
运行时间 all :5.00 min 27 s
test3_ch 53   (54, 253124) (432,)
运行时间 all :5.00 min 51 s
test3_ch 56   (54, 253124) (432,)
运行时间 all :6.00 min 19 s
test3_ch 59   (54, 253124) (432,)
运行时间 all :6.00 min 44 s


In [50]:
import DasBatch as db
import glob
import os
import scipy
def channel_2_wav(flist , save_path , fs ,  start_time = None  ,end_time =None ):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for fname in flist:

        with h5py.File(fname, 'r') as f:
            time_list = [key for key in f.keys()]
            print(time_list[:8])
            if start_time is not None and end_time is not None : 
                    time_list = time_list[time_list.index(start_time):time_list.index(end_time)+1]
            data= [f[key][:] for key in time_list][:]
            data = np.concatenate(data)
            ch = fname.split('/' )[-1][:-5]
            scipy.io.wavfile.write(save_path+ch+".wav" , fs ,data)

channel_2_wav(flist=["../output/DAS/das_as_channel_5_14/test3_ch 26.hdf5"],
              start_time='2024-01-05 01:07:42',
              end_time='2024-01-05 01:37:42',
              fs= 150000,
              save_path= "../output/DAS/das_as_channel_5_14/wav/")   

['2024-01-04 15:00:12', '2024-01-04 15:02:42', '2024-01-04 15:05:12', '2024-01-04 15:07:42', '2024-01-04 15:10:12', '2024-01-04 15:12:42', '2024-01-04 15:15:12', '2024-01-04 15:17:42']


一些需要关注的
从hdf5中读取需要看看是分通道保存还是分时间保存，
计算时选取整10分钟的点位开始，计算时长也要符合，以方便画图
其次，作图时应该考虑分天数画图，如果超过36小时


In [24]:
import h5py 
import numpy as np
import matplotlib.pyplot as plt
import os
def channel_chunk_show_psd_Hz_time_split(psd_path ,ch_list, frequencise, title ,save_path , timeshift = None , split = None , time_ticks = 15):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for ch in ch_list:
        psd = []
        all_time = []
        with h5py.File(psd_path+str(ch)+'.hdf5', 'r') as f:
            
            time = [key for key in f.keys()]
            #日期的排序和字符排序问题
            if timeshift is not None:
                start = time.index(timeshift)
                time = time[start:]+time[:start]
            psd.append([f[key][:] for key in time])
            all_time.append(time[:])
            f.close()
        print("-----------read",ch,"------------")
        psd = np.concatenate(psd)
        time = np.concatenate(all_time)
        print(psd.shape)
        flim= len(psd[0])//2+2
        psd = psd[: , :flim]
        
        psd = np.log(psd)
        print(psd.shape)

        psdlist = np.array_split(psd , split ,axis=0)
        timelist = np.array_split(time ,split)

        for i in range(split):
            psdi = psdlist[i]
            timei = timelist[i]
            
            plt.figure(figsize=(30,10))
            plt.imshow(psdi.T, aspect='auto', cmap='jet',vmin=0, vmax=12)
            plt.grid(alpha = 1)

            timei_show = range(0 , len(timei) , time_ticks )
            
            plt.xticks(timei_show , [timei[t][-8:] for t in timei_show],rotation = 0)

            plt.yticks(np.linspace(0 , len(psdi[0])  , 11 ), np.linspace(0,fs/2 , 11 ,dtype=np.int16))
            plt.ylabel("Frequency(Hz)")
            plt.xlabel("Time")
            plt.title(title+" Channel="+str(ch))

            plt.colorbar()
            plt.savefig(save_path+'ch_'+str(ch)+" " +str(i)+'_psd_.png') #10s per pic
            plt.close()

fs = 1000
db.channel_chunk_show_psd_Hz_time_split(psd_path='../output/DAS/psd_5_12/test2_ch',
                 ch_list = range(1200,1841,20),
                 frequencise=[fs/4],
                 timeshift="12-30 21:55",
                 save_path='../output/DAS/psd_5_12/fig2/' ,
                 split=2,
                 title= "czyc test2 nfft = 30000")

-----------read 1200 ------------
(667, 15001)
(667, 7502)
-----------read 1220 ------------
(667, 15001)
(667, 7502)
-----------read 1240 ------------
(667, 15001)
(667, 7502)
-----------read 1260 ------------
(667, 15001)
(667, 7502)
-----------read 1280 ------------
(667, 15001)
(667, 7502)
-----------read 1300 ------------
(667, 15001)
(667, 7502)
-----------read 1320 ------------
(667, 15001)
(667, 7502)
-----------read 1340 ------------
(667, 15001)
(667, 7502)
-----------read 1360 ------------
(667, 15001)
(667, 7502)
-----------read 1380 ------------
(667, 15001)
(667, 7502)
-----------read 1400 ------------
(667, 15001)
(667, 7502)
-----------read 1420 ------------
(667, 15001)
(667, 7502)
-----------read 1440 ------------
(667, 15001)
(667, 7502)
-----------read 1460 ------------
(667, 15001)
(667, 7502)
-----------read 1480 ------------
(667, 15001)
(667, 7502)
-----------read 1500 ------------
(667, 15001)
(667, 7502)
-----------read 1520 ------------
(667, 15001)
(667, 750

In [25]:
fs = 1000
channel_chunk_show_psd_Hz_time_split(psd_path='../output/DAS/psd_5_11/test2_ch',
                 ch_list = range(1200,1841,20),
                 frequencise=[fs/4],
                #  timeshift="12-30 21:55",
                 save_path='../output/DAS/psd_5_11/fig3/' ,
                 split=2,
                 title= "czyc test2 ")

-----------read 1200 ------------
(564, 15001)
(564, 7502)
-----------read 1220 ------------
(564, 15001)
(564, 7502)
-----------read 1240 ------------
(564, 15001)
(564, 7502)
-----------read 1260 ------------
(564, 15001)
(564, 7502)
-----------read 1280 ------------
(564, 15001)
(564, 7502)
-----------read 1300 ------------
(564, 15001)
(564, 7502)
-----------read 1320 ------------
(564, 15001)
(564, 7502)
-----------read 1340 ------------
(564, 15001)
(564, 7502)
-----------read 1360 ------------
(564, 15001)
(564, 7502)
-----------read 1380 ------------
(564, 15001)
(564, 7502)
-----------read 1400 ------------
(564, 15001)
(564, 7502)
-----------read 1420 ------------
(564, 15001)
(564, 7502)
-----------read 1440 ------------
(564, 15001)
(564, 7502)
-----------read 1460 ------------
(564, 15001)
(564, 7502)
-----------read 1480 ------------
(564, 15001)
(564, 7502)
-----------read 1500 ------------
(564, 15001)
(564, 7502)
-----------read 1520 ------------
(564, 15001)
(564, 750

In [ ]:
# import DasBatch as db
# path = '../output/DAS/stft_5_12/fig/'
# if not os.path.exists(path):
#     os.makedirs(path)

# flist = glob.glob('../output/DAS/stft_5_12/test3_ch*.hdf5')
# flist.sort()
# print(len(flist))
# db.show_stft_time(flist=flist , 
#                ft='../output/DAS/stft_5_12/test3_ft.hdf5',
#                ch_list= range(0,2941 , 20) ,
#                start_time= " " , 
#                end_time=" " , 
#                title= "czyc test2  ",
#                save_path=path)

hi，有个新的想法。把创智云城psd做一个傅里叶反变换，就是自相关函数。然后画出来自相关函数随通道变化的热度图，可以试试能不能看地下结构信息